In [ ]:
!pip install transformers

In [ ]:
import os
from google.colab import drive

import transformers
import torch
import datasets

import pandas as pd

## **Caricamento dei dati**

Importo il mio *Google Drive* per poter accedere alla cartella contenente i dati del task.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_folder = "/content/drive/My Drive/profiling_input"
documents = os.listdir(data_folder)

train_data = []
test_data = []

# Creo una lista con le labels, per identificare quante sono e passare successivamente l'informazione direttamente al modello
labels = []

for document in documents:
  # Seleziono come label 'TOPIC'
  label = document.rstrip('.txt').split('#')[2]

  if not label in labels:
    labels.append(label)

  type_document = document.rstrip('.txt').split('#')[0]
  data = open('/content/drive/My Drive/profiling_input/' + document, 'r')
  text = data.read()

  input_instance = {}
  input_instance["id"] = document
  input_instance["sentence"] = text
  input_instance["label"] = label

  if type_document == "training":
    train_data.append(input_instance)
  else:
    test_data.append(input_instance)

# Creo un dizionario di mapping fra le labels come stringhe e le label numeriche
str_to_int = {label: num for num, label in enumerate(labels)}



Preprocessing dei dati.

I dati vengono prima convertiti nel formato datasets e poi il dataset di training viene ulteriormente diviso in train e validation set.


In [ ]:
# Conversione da lista di dizionari a dataset
train = datasets.Dataset.from_pandas(pd.DataFrame(data=train_data))
test = datasets.Dataset.from_pandas(pd.DataFrame(data=test_data))

# Splitting in train e dev del training set
train_dev = train.train_test_split(test_size=0.1)

train = train_dev["train"]
dev = train_dev["test"]

# Stampo 'train' per vedere come è formattato
print("Esempio di testo:", train['sentence'][2])
print()
print("Esempio di label:", train['label'][2])

Esempio di testo: Credo che ci sarò anche io
Ecco le mie foto:
Un grande grazie a Filippo e a tutti per questo giro, mi è piaciuto molto, conoscevo la parte iniziale ma non credo di essere stata prima al valico della Somma. Primo giro di primavera con questa splendida giornata, che mi ha trovata un pò sottoallenata e ancora tossicchiante dopo l'influenza di febbraio... Vi metto in guardia, sto risalendo con il mio allenamento e la prossima volta partirò per prima sollevando la polvere dietro di me              Ancora tanti auguri a Maria  Vi dedico questa frase (sotto) di Jack Kerouac, che dedico anche a tutti gli altri miei compagni di pedale, è un privilegio pedalare con voi tutti, anche se a volte qualcosa può andare storto  «Alla fine non ricorderai il tempo passato in ufficio o a pettinare il giardino. Perciò scala quella maledetta montagna» (Jack Kerouac)
Ci sono anch'io, salirò a Tiburtina, suppongo che il treno che devo prendere sia quello delle 8:07 a Tiburtina (che dovrebbe e

## **Caricamento del modello** ##

Di seguito, tramite la libreria *transformers* di Huggingface, carichiamo il nostro Transformer model.
A differenza dei notebook precedenti, in questo caso carichiamo il modello tramite la *pipeline* di Huggingface.

Le pipeline oggetti che astraggono la maggior parte del codice dalla libreria, offrendo una semplice API dedicata a diversi compiti, tra cui Named Entity Recognition, (Masked) Language Modeling, Sentiment Analysis, Feature Extraction e Question Answering.

In [ ]:
model_id = "swap-uniba/LLaMAntino-3-ANITA-8B-Inst-DPO-ITA"

# Initialize the pipeline.
pipeline = transformers.pipeline(
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu


Dopo aver caricato il modello, possiamo predisporre il testo di input che andremo a passare al modello. Utilizzando la pipeline di Huggingface, non c'è bisogno di utilizzare prima il tokenizzatore: la pipeline si occuperà automaticamente sia della tokenizzazione che della generazione.

L'input deve essere strutturato con la seguente struttura:


*   *role*: si riferisce generalmente a una etichetta assegnata a ciascun messaggio all'interno di una sequenza conversazionale. Quando settato su *user* rappresenta l’input dell’utente, ovvero la richiesta o la domanda che viene posta al modello.
*   *content*: testo (e.g. frase, documento) da passare al modello



In [ ]:
# Input text for the model.
input_conv = [{"role": "user", "content": "Qual è la capitale dell'Italia?"}]

A questo punto possiamo utilizzare la pipeline per generare la risposta del modello.

In [ ]:
output = pipeline(
  input_conv,
  max_new_tokens=128,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(output)
print()
print(output[0]["generated_text"][1]["content"])

[{'generated_text': [{'role': 'user', 'content': "Qual è la capitale dell'Italia?"}, {'role': 'assistant', 'content': "La capitale dell'Italia è Roma."}]}]

La capitale dell'Italia è Roma.


## **Classificazione con ANITA** ##

Proviamo ad utilizzare Minerva per classificare uno dei documenti estratti da TAG-IT.
Per poterlo fare, dobbiamo costruire un prompt che specifichi al modello che task deve risolvere e quali sono le possibili etichette da predire.

In [ ]:
sentences = train['sentence'][100].split("\n")[:10]
document = "\n".join(sentences)

prompt = "Ti fornirò un documento. Il tuo compito è leggerlo e assegnargli una sola etichetta tra le seguenti categorie:\n" + str(labels)
prompt += "\nRestituisci solo l'etichetta più adatta. Non fornire spiegazioni\nDocumento:"
prompt += document

print(prompt)

Ti fornirò un documento. Il tuo compito è leggerlo e assegnargli una sola etichetta tra le seguenti categorie:
['SPORTS', 'AUTO-MOTO', 'METAL-DETECTING', 'ENTERTAINMENT', 'NATURE', 'ANIME', 'SMOKE', 'CELEBRITIES', 'BIKES', 'MEDICINE-AESTHETICS', 'TECHNOLOGY']
Restituisci solo l'etichetta più adatta. Non fornire spiegazioni
Documento:esatto, proprio cosi!
Grazie dei complimenti! Quella misura era riportata sul vecchio pneumatico che montava all'anteriore. Il diametro interno è perfetto, la spalla secondo me è un pelo bassa ma ci può stare. Comunque non sto smettendo di cercare quelle giuste.
Aggiornamento pneumatici. Ho montato le 24 1-3/4, calzano perfettamente ed il disegno del battistrada non è male. Il motore l'ho aperto, messo seeger mancante e richiuso. Ho fatto qualche giro di prova ma la frizione slitta. Il gioco c'è, l'olio è giusto ed i dischi frizione sono buoni. Rimane da sostituire le molle. Qualcuno sa dove reperirle? anche non originali ma compatibili ...nuove ovviamente


In [ ]:
input_conv = [{"role": "user", "content": prompt}]

output = pipeline(
  input_conv,
  max_new_tokens=128,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(output)
print()
print("Output Minerva:", output[0]["generated_text"][1]["content"])
print("Etichetta gold:", train['label'][2])

[{'generated_text': [{'role': 'user', 'content': 'Ti fornirò un documento. Il tuo compito è leggerlo e assegnargli una sola etichetta tra le seguenti categorie:\n[\'SPORTS\', \'AUTO-MOTO\', \'METAL-DETECTING\', \'ENTERTAINMENT\', \'NATURE\', \'ANIME\', \'SMOKE\', \'CELEBRITIES\', \'BIKES\', \'MEDICINE-AESTHETICS\', \'TECHNOLOGY\']\nRestituisci solo l\'etichetta più adatta. Non fornire spiegazioni\nDocumento:esatto, proprio cosi!\nGrazie dei complimenti! Quella misura era riportata sul vecchio pneumatico che montava all\'anteriore. Il diametro interno è perfetto, la spalla secondo me è un pelo bassa ma ci può stare. Comunque non sto smettendo di cercare quelle giuste.\nAggiornamento pneumatici. Ho montato le 24 1-3/4, calzano perfettamente ed il disegno del battistrada non è male. Il motore l\'ho aperto, messo seeger mancante e richiuso. Ho fatto qualche giro di prova ma la frizione slitta. Il gioco c\'è, l\'olio è giusto ed i dischi frizione sono buoni. Rimane da sostituire le molle. Q